# Imports

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
from scipy import stats
from scipy.linalg import solveh_banded
from sklearn.decomposition import PCA
from collections import OrderedDict
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse
import matplotlib.gridspec as gridspec
import scipy
from bioinfokit.analys import stat
import scipy.stats as stats
import seaborn as sn
from statannot import add_stat_annotation
from scipy.signal import savgol_filter 
from itertools import chain
from minisom import MiniSom
import matplotlib
from sklearn import preprocessing
from matplotlib.patches import RegularPolygon 
import seaborn as sns

# Parameters

Folder inputs

there are 3 different formattings:

1) r'/...'
2) r'C:\...'
3) 'C:\\...'

But basically just copy past the directory from your main folder. Since I am using ubuntu it comes out as the 1) option. On windows it should be option 2)

The data should be organized in a hierarchical way, ie, samples > replicates > single spectra.txt (broken from a Raman map using the Renishaw software)

In [2]:
mainfolders = [
        r'/home/paulo/Documents/PhD/own papers/mike paper/all data/Corynebacterium glutamicum'
        ,
        r'/home/paulo/Documents/PhD/own papers/mike paper/all data/Mycobacterium bovis BCG'
        ,
        r'/home/paulo/Documents/PhD/own papers/mike paper/all data/Rhodococcus erythropolis'
        ]

if all(len(a.split('\\'))>1 for a in mainfolders):
    [('/').join(a.split('\\')) for a in mainfolders]
    
#colour setup (still needs to be coded)
NUM_COLORS = len(mainfolders)  
cmaps = plt.get_cmap('gist_rainbow')
colours = [cmaps(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

#if you want a specific colour scheme write it here and comment the previous lines
# check the colour list in https://matplotlib.org/2.2.0/gallery/color/named_colors.html
#colours = []

import matplotlib as mpl
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colours) 

#the clean parameter is related to a z-score test sigma value. A value of 3 is enought to clean up the data
# if the data values are outliers they will be removed.
clean = 3

#save folder (to save the excel sheet related to the peak finding script)
savefolder = r'/home/paulo/Documents/PhD/own papers/mike paper/all data/peaks/'

#SOM section insert the number of hexagon sides that you want (square shape)
neuron = 4

Plot setup

In [3]:
font = {'family' : 'Arial',
        'size'   : 16}

plt.rc('font', **font)

%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [9.0, 9.0/1.618]
mpl.rcParams['figure.dpi'] = 50

# Scripts

In [4]:
# From MatrixExp
def matrix_exp_eigen(U, s, t, x):
    exp_diag = np.diag(np.exp(s * t), 0)
    return U.dot(exp_diag.dot(U.transpose().dot(x))) 

# From LineLaplacianBuilder
def get_line_laplacian_eigen(n):
    assert n > 1
    eigen_vectors = np.zeros([n, n])
    eigen_values = np.zeros([n])

    for j in range(1, n + 1):
        theta = np.pi * (j - 1) / (2 * n)
        sin = np.sin(theta)
        eigen_values[j - 1] = 4 * sin * sin
        if j == 0:
            sqrt = 1 / np.sqrt(n)
            for i in range(1, n + 1):
                eigen_vectors[i - 1, j - 1] = sqrt
        else:
            for i in range(1, n + 1):
                theta = (np.pi * (i - 0.5) * (j - 1)) / n
                math_sqrt = np.sqrt(2.0 / n)
                eigen_vectors[i - 1, j - 1] = math_sqrt * np.cos(theta)
    return eigen_vectors, eigen_values

def smooth2(t, signal):
    dim = signal.shape[0]
    U, s = get_line_laplacian_eigen(dim)
    return matrix_exp_eigen(U, -s, t, signal)


def als_baseline(intensities, asymmetry_param=0.05, smoothness_param=1e6,
                 max_iters=10, conv_thresh=1e-5, verbose=False):
  '''Computes the asymmetric least squares baseline.
  * http://www.science.uva.nl/~hboelens/publications/draftpub/Eilers_2005.pdf
  smoothness_param: Relative importance of smoothness of the predicted response.
  asymmetry_param (p): if y > z, w = p, otherwise w = 1-p.
                       Setting p=1 is effectively a hinge loss.
  '''
  smoother = WhittakerSmoother(intensities, smoothness_param, deriv_order=2)
  # Rename p for concision.
  p = asymmetry_param
  # Initialize weights.
  w = np.ones(intensities.shape[0])
  for i in range(max_iters):
    z = smoother.smooth(w)
    mask = intensities > z
    new_w = p*mask + (1-p)*(~mask)
    conv = np.linalg.norm(new_w - w)
    if verbose:
      print (i+1, conv)
    if conv < conv_thresh:
      break
    w = new_w
  else:
    print ('ALS did not converge in %d iterations' % max_iters)
  return z


class WhittakerSmoother(object):
  def __init__(self, signal, smoothness_param, deriv_order=1):
    self.y = signal
    assert deriv_order > 0, 'deriv_order must be an int > 0'
    # Compute the fixed derivative of identity (D).
    d = np.zeros(deriv_order*2 + 1, dtype=int)
    d[deriv_order] = 1
    d = np.diff(d, n=deriv_order)
    n = self.y.shape[0]
    k = len(d)
    s = float(smoothness_param)

    # Here be dragons: essentially we're faking a big banded matrix D,
    # doing s * D.T.dot(D) with it, then taking the upper triangular bands.
    diag_sums = np.vstack([
        np.pad(s*np.cumsum(d[-i:]*d[:i]), ((k-i,0),), 'constant')
        for i in range(1, k+1)])
    upper_bands = np.tile(diag_sums[:,-1:], n)
    upper_bands[:,:k] = diag_sums
    for i,ds in enumerate(diag_sums):
      upper_bands[i,-i-1:] = ds[::-1][:i+1]
    self.upper_bands = upper_bands

  def smooth(self, w):
    foo = self.upper_bands.copy()
    foo[-1] += w  # last row is the diagonal
    return solveh_banded(foo, w * self.y, overwrite_ab=True, overwrite_b=True)


def listdirs(rootdir):
    classes = os.listdir(rootdir)
    subdires = []
    files = []
    for file in classes:
        subdir = os.path.join(rootdir, file)
        subdires.append(subdir)
        if os.path.isdir(subdir):
            files.append(listdirs(subdir)[1])
          
    allfiles = [item for sublist in files for item in sublist]
    
    return classes,subdires,allfiles



def cleanfile(files):
    ofiles = []
    for file in files:
        for raw in file[2]:
            for r in raw[1]:
                if r != []:
                    ofiles.append(r)
                    
    return ofiles



def fixwave(spec,wave):
    minmin = wave.min(axis=1).min()+10
    maxmax = wave.max(axis=1).max()-10
    xx = np.linspace(minmin,maxmax,wave.shape[1])
    data = []
    
#    spec = spec.dropna(axis=0)
#    wave = wave.dropna(axis=0)
    
    for spe,wav in zip(spec.values, wave.values):
        spl1 = make_interp_spline(sorted(wav),spe[::-1], k=3)  # type: BSpline
        data.append(spl1(xx)[::-1])
    
    return pd.DataFrame(data,columns = xx[::-1],index=spec.index)

def analyse_data_folders(mainfolders,clean):
    wave = pd.DataFrame()
    spec = pd.DataFrame()
    
    for mainfolder in mainfolders:
            
        label = mainfolder.split("/")[-1]
        classes, subdires,files = listdirs(mainfolder)
        
        #files = cleanfile(files)   
        
        
        data = []
        
        for file in files:
            df = pd.read_csv(file,sep='\t', header= None)
            df.columns = ['#Wave','#Intensity']
            df['#Intensity'] = df['#Intensity'].values - als_baseline(df['#Intensity'].values)
            df['#Intensity'] = (df['#Intensity']-df['#Intensity'].min())/(df['#Intensity'].max()-df['#Intensity'].min())   
            data.append(df.T)
            
        alldf = pd.concat(data,axis=0)
        
        
        
        # separate wavenumber and intensity and remove nan
        allwave = alldf[alldf.index.values=='#Wave'].iloc[:,:887]
        allint0 =  alldf[alldf.index.values=='#Intensity'].iloc[:,:887].dropna(axis=0)
#                         .dropna(axis=1)
                         
        allwave = allwave.iloc[:allint0.shape[0],:]
                       
    
        
        # use zscore to remove outliers from dataframe
        allint  = allint0[(np.abs(stats.zscore(allint0)) < clean).all(axis=1)]
        allint.index = [label]*allint.shape[0]
        allwave  = allwave[(np.abs(stats.zscore(allint0.values)) < clean).all(axis=1)]
        
        spec = pd.concat([spec,allint])
    #    spec.append(allint)
        wave = pd.concat([wave,allwave])


    return spec,wave
    
def plot_avg(finaldf):
        
    plt.figure(figsize=(9,9/1.618))
    for cnt, (l,c) in enumerate(zip(finaldf.index.unique(),['r','g','b'])):
        region = finaldf.index == l
        plt.plot(finaldf.columns, finaldf.values[region].mean(axis=0)+cnt, c = c, label=l, lw=1.5)
        plt.fill_between(finaldf.columns, finaldf.values[region].mean(axis=0)+cnt,cnt+finaldf.values[region].mean(axis=0)+finaldf.values[region].std(axis=0),color='k',alpha=0.5)
    
    plt.legend(loc='best', frameon=False,prop={'size': 12})
    plt.xlabel('Raman shift (cm$^{-1}$)')
    plt.ylabel('Relative Intensity (a.u.)')



def OrganizePCAData(norm,labels,wavenumber):    
    df_pca = pd.DataFrame(norm).reset_index(drop=True)
    target = pd.DataFrame(labels,columns=['sample'])
    
    pca = PCA()
    principalComponents = pca.fit_transform(df_pca)
    columns = ['principal component '+str(i+1) for i in range(principalComponents.shape[1])]
    info = ['PC '+str(i+1)+': '+str(round(pca.explained_variance_ratio_[i]*100,2))+' % \n ' for i in range(principalComponents.shape[1])]
    principalDf = pd.DataFrame(data = principalComponents , columns = columns)
    df = pd.concat([principalDf, target], axis = 1)

    return df,target,info,pca,wavenumber



def pcafit(norm, wavenumber, labels):
    df_pca = pd.DataFrame(norm).reset_index(drop=True)
    target = pd.DataFrame(labels,columns=['sample'])
    
    pca = PCA()
    principalComponents = pca.fit_transform(df_pca)
    columns = ['principal component '+str(i+1) for i in range(principalComponents.shape[1])]

    principalDf = pd.DataFrame(data = principalComponents , columns = columns)
    df = pd.concat([principalDf, target], axis = 1)
   
    return pca,wavenumber,df



def find_local_max(signal):
    dim = signal.shape[0]
    ans = np.zeros(dim)
    for i in range(dim):
        dxf = signal[i - 1] - signal[i]
        dxb = signal[i - 1] - signal[i - 2]
        ans[i] = 1.0 if dxf > 0 and dxb > 0 else 0.0
    return ans > 0.5

def fit_region(wavenumber,pos,tol = 10):
    min_l = min(wavenumber.index.tolist())
    max_l = max(wavenumber.index.tolist())
    index = wavenumber.index[wavenumber==pos].tolist()[0]
    index1 = index-tol
    index2 = index+tol
    if index1<min_l:
        index1 = 0
    if index2>max_l:
        index2 = max_l
        
    return wavenumber[index1:index2]

def linear_fit(x,a,b):
    return a*x+b

def power_law(x, a, b):
    return a*np.power(x, b)

def _1Lorentzian(x, amp, cen, wid):
    return amp*wid**2/((x-cen)**2+wid**2)


def get_stats(finaldf,peaks):
    
    data = []
    label = []
    thick = []

    for lab in finaldf.index.unique():
        for p in peaks:
                
            intensity = finaldf[finaldf.index==lab]
            
            label.append(lab)
            
            thick.append(p)
            
            
            data.append(np.max(intensity.values[:,(intensity.columns.values>p-20) & (intensity.columns.values<p+20)],axis=1))
        
        
    df = pd.DataFrame(data).dropna(axis=1)
    label = pd.DataFrame(label)
    thick = pd.DataFrame(thick)
    
    finaldf = pd.concat([df,label,thick],axis=1).reset_index(drop=True)
    finaldf.columns = np.array(list(range(0,finaldf.shape[1]-2))+['sample','peak'])
    finaldf = finaldf.sort_values(['peak','sample'])
    
    x = 'peak'
    y = 'value'
    hue = 'sample'
    #    order = ['st95 mel','st95']
    melt = pd.melt(finaldf,id_vars=[hue,x]) 
    fig,ax = plt.subplots(figsize=(9,9/1.618))
    #    ax = sns.boxplot(x=x, y=y, data=df_melt,palette=['#00FFFF','#0000FF'])
    ax = sns.boxplot(x = x , 
                     y = y , 
                     hue = hue ,
                     data = melt, 
                     palette = ['r','g','b'])
#                     palette = ['#00FFFF','#0000FF'])
    #                 palette = ['#FF8000','#FF0000'])
                                
    #    ax.set_title('peak = ' + str(t) +' cm$^{-1}$')
    
    add_stat_annotation(ax, data = melt,
                        x=x,
                        y=y,
                        hue=hue,
                    box_pairs=[((p,"Corynebacterium glutamicum"),(p,"Mycobacterium bovis BCG")) for p in peaks]+[((p,"Corynebacterium glutamicum"),(p,"Rhodococcus erythropolis")) for p in peaks]+[((p,"Mycobacterium bovis BCG"),(p,"Rhodococcus erythropolis")) for p in peaks],
                    test='t-test_ind', text_format='star', loc='inside', verbose=2)
    plt.legend(loc='best', frameon=False)
    #    ax.set_yticklabels(np.linspace(0,1,11))
    #    ax.set_xticklabels(np.linspace(500,1600,23))
    ax.set_xlabel('Raman shift (cm$^{-1}$)')
    ax.set_ylabel('Rel. Intensity (a.u.)')
    
   
    

def LoadingsPlot(wavenumber,pca):
    plt.figure(figsize=(9,9/1.618))
    loadings = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_))
    loadings = loadings.iloc[:,:2]
    loadings.columns = ['LPC1','LPC2']
    plt.plot(wavenumber,savgol_filter(loadings['LPC1'],11,3),label='LPC1')
    plt.plot(wavenumber,savgol_filter(loadings['LPC2'],11,3),ls='--',label='LPC2')
    plt.legend(loc='best',frameon=False)
    plt.xlabel('Raman shift (cm$^{-1}$)')
    plt.ylabel('Loadings')

  

def plotload(pca,wavenumber):
    loadings = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_))

    loadings = loadings.iloc[:,:2]
    loadings.columns = ['LPC1','LPC2']


    return loadings



def smooth3(t, signal):
    signal_i = signal[0]
    dim = signal.shape[0]
    U, s = get_line_laplacian_eigen(dim)
    return matrix_exp_eigen(U, -s, t, signal)-(matrix_exp_eigen(U, -s, t, signal)[0]-signal_i)


def plotprinc(df,loadings,wavenumber,pca,smooth):
    
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color'] 
    
    colours = []
    
    for i in range(4):
        colours.append(cycle[i%len(cycle)])
    

    m1 =[1,-1,-1,1]
    m2 = [1,1,-1,-1]
    label = [['PC1P','PC2P'],['PC1N','PC2P'],['PC1N','PC2N'],['PC1P','PC2N']]
        
    fig, axs = plt.subplots(4, 4,figsize=(12,12/1.618))
    gs = axs[1, 1].get_gridspec()  
#    axs[2,2].remove()
    for ax in list(chain(*axs[:2,:2])):
        ax.remove()
        
    for ax in list(chain(*axs[2:,:2])):
        ax.remove()
        
    for j in range(0,4):
        for ax in axs[j,2:]:
            ax.remove()
#            for ax in axs[2:,j]:
#                ax.remove()

        
    axbig = fig.add_subplot(gs[:2,:2])
    axbig2 = fig.add_subplot(gs[2:,:2])
    
    axsma = fig.add_subplot(gs[0,2:])
    axsma2 = fig.add_subplot(gs[1,2:])
    
    axsma3 = fig.add_subplot(gs[2,2:])
    axsma4 = fig.add_subplot(gs[3,2:])
    
    combax = [axsma,axsma2,axsma3,axsma4]
 
    axbig2.plot(wavenumber,loadings['LPC1'],label = 'LPC1')
    axbig2.plot(wavenumber,loadings['LPC2'],label = 'LPC2')
    axbig2.legend(loc='best', frameon=False,prop={'size': 10})  
    
    axbig.grid(True)
    axbig2.grid(True)
    
    NUM_COLORS = len(df['sample'].unique().tolist())
    
    cmaps = plt.get_cmap('gist_rainbow')
    
    colours2 = [cmaps(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    

    for target,colo in zip(df['sample'].unique().tolist(),colours2):
        index = df['sample'] == target
        axbig.plot(df['principal component 1'][index],df['principal component 2'][index],'o',label=target,c=colo)
    
    axbig.fill_between(np.linspace(0, df['principal component 1'].max()*1.1, 10),0,(df['principal component 2'].max()*1.1),alpha=0.2)
    axbig.fill_between(np.linspace(df['principal component 1'].min()*1.1,0, 10),0,(df['principal component 2'].max()*1.1),alpha=0.2)
    axbig.fill_between(np.linspace(df['principal component 1'].min()*1.1,0, 10),0,(df['principal component 2'].min()*1.1),alpha=0.2)
    axbig.fill_between(np.linspace(0, df['principal component 1'].max()*1.1, 10),0,(df['principal component 2'].min()*1.1),alpha=0.2)
           
    
    axbig.legend(loc='best', frameon=False,prop={'size': 8})
    axbig.set_xlabel('PC1 : '+str(round(pca.explained_variance_ratio_[0]*100,2))+'%')
    axbig.set_ylabel('PC2 : '+str(round(pca.explained_variance_ratio_[1]*100,2))+'%')

    axbig2.set_xlabel('Raman shift (cm$^{-1}$)')
    axbig2.set_ylabel('Loadings')
    
    maxlod = loadings.max().max()
    
    axbig2.fill_between(wavenumber,maxlod*1.1,alpha=0.5,color='dimgrey')
    axbig2.fill_between(wavenumber,-maxlod*1.1,alpha=0.5,color='gainsboro')
    
    

    x = wavenumber
    y1 = smooth3(smooth*2,loadings['LPC1'])
    y2 = smooth3(smooth*2,loadings['LPC2'])
     
    i = 0 

    peaksQ = []
    
    for m_1,m_2 in zip(m1,m2):
        
#        ax[i] = fig.add_subplot()
        
        y1_m = y1*m_1
        y2_m = y2*m_2
        
        local_max_index2 = (y1_m>0) & (find_local_max(y1_m))
        stem2 = np.zeros(x.shape)
        stem2[local_max_index2] = 1 
    
#        local_max_index3 = s2(y2,0) & (find_local_max(y2))
        local_max_index3 = (y2_m>0) & (find_local_max(y2_m))
        stem3 = np.zeros(x.shape)
        stem3[local_max_index3] = 1 
        
#        plt.figure()
        combax[i].plot(x[y1_m>0],y1_m[y1_m>0],'o')
        combax[i].plot(x[y2_m>0],y2_m[y2_m>0],'o')
        combax[i].plot(x[local_max_index2],y1_m[local_max_index2],'rx')
        combax[i].plot(x[local_max_index3],y2_m[local_max_index3],'rx')
        combax[i].text(x.min()*1.1,y1_m.max()*0.7,'Q'+str(i+1),color=colours[i])

        combax[i].set_xlabel('Raman shift (cm$^{-1}$)')
        combax[i].set_ylabel('L(a.u.)')
        

        
        for wav in wavenumber[local_max_index2]:
            reg = fit_region(wavenumber,wav,10)
            
            if (y1_m[reg[reg==wav].index][0] > 0):
                try:
                    popt_1lorentz, pcov_1lorentz = scipy.optimize.curve_fit(_1Lorentzian, 
                                                                    reg, 
                                                                    y1_m[reg.index],
                                                                    p0=[y1_m[reg.index].max(), wav, 2/(np.pi*y1_m[reg.index].max())])
                    pars_1 = popt_1lorentz[0:3]
                    
                    if (abs(pars_1[-1])>100) | (pars_1[0]<0):
                        pass
                    else:
                        
                        combax[i].plot(wavenumber,_1Lorentzian(wavenumber,*popt_1lorentz),lw=2)
                        combax[i].fill_between(wavenumber, _1Lorentzian(wavenumber,*popt_1lorentz).min(),_1Lorentzian(wavenumber,*popt_1lorentz), alpha=0.5)
                        peaksQ.append([df['sample'][0]]+popt_1lorentz.tolist()+['Q'+str(i+1)])
    
                except:
                    pass

                
        for wav in wavenumber[local_max_index3]:
            reg = fit_region(wavenumber,wav,10)
            
            if (y2_m[reg[reg==wav].index][0] > 0):
    
                try:
                    popt_1lorentz, pcov_1lorentz = scipy.optimize.curve_fit(_1Lorentzian, 
                                                                    reg, 
                                                                    y2_m[reg.index],
                                                                    p0=[y2_m[reg.index].max(), wav, 2/(np.pi*y2_m[reg.index].max())])
                    
       
                    pars_1 = popt_1lorentz[0:3]
                    if (abs(pars_1[-1])>100) | (pars_1[0]<0):
                        pass
                    else:

                        combax[i].plot(wavenumber,_1Lorentzian(wavenumber,*popt_1lorentz),lw=2)
                        combax[i].fill_between(wavenumber, _1Lorentzian(wavenumber,*popt_1lorentz).min(),
                                         _1Lorentzian(wavenumber,*popt_1lorentz), alpha=0.5)

                        peaksQ.append([df['sample'][0]]+popt_1lorentz.tolist()+['Q'+str(i+1)])
                    
                except:
                    pass   
                
        i = i + 1
        
        peak = pd.DataFrame(peaksQ,columns=['label','height','center','width','importance'] )       
         
    peak.to_csv('loadings'+df['sample'][0]+'.csv',sep=';', index=False)

    
    
    plt.legend(loc='best', frameon=False,prop={'size': 8})    
    fig.tight_layout()
#    plt.grid(True)
    plt.show()



def loadingpeak(finaldf):
    
    smooth = 7
    norm = finaldf.values
    wavenumber = pd.Series(finaldf.columns.values)
    labels = finaldf.index.values
    
    pca,wavenumber,df= pcafit(norm, wavenumber, labels)
    loadings = plotload(pca,wavenumber)
    plotprinc(df,loadings,wavenumber,pca,smooth)
    


def LoadSOM(data,neuron):

    dataval = data.values
    
    som = MiniSom(neuron,neuron, data.shape[1], sigma=1.5, learning_rate=.5, 
                  neighborhood_function='gaussian', random_seed=0)
    
    som.pca_weights_init(dataval)
    som.train(dataval, neuron*3000, verbose=True)  # random training

    return som 


def PlotAvgSpec(data):
    classes = np.unique(data.index)
    colours = ['r','g','b']
    
    plt.figure(figsize=(9,9/1.618))
    for cnt, (lab,col) in enumerate(zip(classes,colours)):
        index = lab == data.index
        xx = data.columns
        yy = data[index].values
        plt.plot( xx , yy.mean(axis=0)+cnt , label = lab,color=matplotlib.colors.to_rgba(col,alpha=1),lw=2)
    
        for cnt2,y in enumerate(yy):

            yavg2 = (y-np.min(np.mean(yy,axis=0)))/(np.max(np.mean(yy,axis=0))-np.min(np.mean(yy,axis=0)))
            plt.plot(xx, cnt + yavg2, color = matplotlib.colors.to_rgba(col,alpha=0.01))
        
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.xlabel('Raman shift (cm$^{-1}$)')
    plt.ylabel('Relative Intensity (a.u.)')
    plt.legend(by_label.values(), by_label.keys(),loc='best', frameon=False,prop={'size': 12})
    plt.show()



def ColorHexBins(plot_points,cla_col,wy,wx,alpha):
    final_c = []
    color_hex = []
    for pp,cc in zip(plot_points,cla_col):
        color_hex.append([pp[0],pp[1],cc[0],cc[1]])
    
    df_color_hex = pd.DataFrame(color_hex,columns=['pos x','pos y','label','color'])
    
    uni = (wx,wy)
    index1 =  df_color_hex['pos x']==uni[0]
    index2 =  df_color_hex['pos y']==uni[1]
    index = index1 & index2

    df_clean = df_color_hex[index].reset_index(drop=True) 
    
    counts = []
    
    for lab in df_clean['label'].unique():
        index3 = df_clean['label'] == lab
        counts.append([sum(index3),lab])

        
    counts = pd.DataFrame(counts,columns=['counts','label'])
    counts = counts.sort_values(by='counts')
    
    if counts.empty or counts['counts'].iloc[-1] == 0:
        final_c.append((0.5,0.5,0.5,1))
    else:
        index4 =  df_color_hex['pos x']==uni[0]
        index5 =  df_color_hex['pos y']==uni[1]
        index6 = df_color_hex['label']==counts['label'].iloc[-1]
        index7 = index4 & index5 & index6
        
        final_c.append(df_color_hex[index7]['color'].unique()[0])

    return matplotlib.colors.to_rgba(final_c[0],alpha=alpha)
        


def HexagonaSOMPlot(som,data,neuron):
    
    colours = ['r','g','b']
    
    columns = list(data.columns)  
#    activation = pd.DataFrame([[a[0],a[1], b[0]] for a,b in som.win_map( data.values).items()])
    somatrix = pd.DataFrame([[a[0],a[1],b.most_common()[0][0],b.most_common()[0][1]] for a,b in som.labels_map(data.values,data.index.values).items()],columns=['row','col','label','counts']).sort_values(by=['row','col']).reset_index(drop=True)

    array = []

    for i in range(neuron):
#        counter = 0
        for j in range(neuron):
            
#            for soma in somatrix.values:
                
#                if (soma[0] == i) and (soma[1] == j):
            if np.any((somatrix['row']==i) & (somatrix['col']==j)):
                array.append(somatrix[(somatrix['row']==i) & (somatrix['col']==j)].values[0])
#                array.append([i,j,soma[2],soma[3]])
#                counter = 1
            else:
                
#            if counter == 0:
                array.append([i,j, np.nan, np.nan])
        
    somatrix = pd.DataFrame(array,columns=somatrix.columns)
                
    
    target = list(data.index.values)
    le = preprocessing.LabelEncoder()
    le.fit(target)

    
    classes = np.unique(data.index)
    
   
    dataval = data.values
    xx, yy = som.get_euclidean_coordinates()
    umatrix = som.distance_map()
    weights = som.get_weights()
    
    
    #code to align the hexagonals
    a = []
    
    for i in range(xx.shape[0]):
        b = []
        for j in range(xx.shape[1]):   
            if j % 2 == 0:
                b.append(0)
            else:
                b.append(0.5)
        a.append(b)
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.set_aspect('equal')
    
       
    #enumerate gives you the count (cnt) and the interation value (x) represents each row
    cla_col = []
            
    plot_points = []
    
    maxcol = []
    for c , color in zip(classes,colours):
        idx_target = data.index.values == c
        
        
        maxcol.append([c,somatrix[somatrix['label']==c]['counts'].max()])
        
        for cnt, x in enumerate(dataval[idx_target]):
            # getting the winner
            w = som.winner(x)
            # place a marker on the winning position for the sample xx
            wx, wy = som.convert_map_to_euclidean(w) 
            
            if wy%2 == 0:
                wx = wx
            else:
                wx = wx+0.5
            
            wy = wy * np.sqrt(3) / 2
            
            plot_points.append([wx,wy])
            cla_col.append([c,color])
    


    
     # iteratively add hexagons
#    plt.figure()
         
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111)
    
    for label,color in zip(classes,colours):         
        index = somatrix['label'] == label
        ax2.plot(columns,
                 np.mean(weights.reshape(weights.shape[0]*weights.shape[1],weights.shape[2])[index],axis=0),
                 c= color,
                 zorder=1000,
                 linewidth = 2)

    
    for i in range(weights.shape[0]):
        for j in range(weights.shape[1]):
            
            if somatrix['label'][i*weights.shape[0]+j] is np.nan:
                wy = yy[(i, j)] * np.sqrt(3) / 2
                wx = xx[(i, j)] + pd.DataFrame(a).values[(i,j)]
                hexa = RegularPolygon((wx, wy), 
                                         numVertices=6, 
                                         radius=.95 / np.sqrt(3),
                                         facecolor=ColorHexBins(plot_points,cla_col,wy,wx,0),
                                         linewidth=1,
                                         edgecolor= (0,0,0,1))
        
                ax.add_patch(hexa)
            
            else:
                    
                wy = yy[(i, j)] * np.sqrt(3) / 2
                wx = xx[(i, j)] + pd.DataFrame(a).values[(i,j)]
                ax2.plot(columns,
                         weights[i][j], 
                         color = matplotlib.colors.to_rgba([b for a,b in zip(classes,colours) if a == somatrix['label'][i*weights.shape[0]+j]][0],alpha=0.05),
                label = somatrix['label'][i*weights.shape[0]+j])
    
                for col in maxcol:
                    lab,con = col
                    if somatrix['label'].values[i*weights.shape[0]+j] == lab:
                        alpha = somatrix['counts'].values[i*weights.shape[0]+j]/con
                        
                        hexa = RegularPolygon((wx, wy), 
                                             numVertices=6, 
                                             radius=.95 / np.sqrt(3),
                                             facecolor=ColorHexBins(plot_points,cla_col,wy,wx,alpha),
                                             linewidth=1,
                                             edgecolor= (0,0,0,1))
            
                        ax.add_patch(hexa)
            
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    leg = ax2.legend(by_label.values(), by_label.keys(),loc='best', frameon=False)
    
    for lh in leg.legendHandles: 
        lh.set_alpha(1)
    
    ax2.set_xlabel('Raman shift (cm$^{-1}$)')
    ax2.set_ylabel('Activations (a.u.)')
    
    
    
    xrange = np.arange(weights.shape[0])
    yrange = np.arange(weights.shape[1])
    ax.set_xlim(xrange.min()-1, xrange.max()+1)
    ax.set_ylim(yrange.min() * np.sqrt(3) / 2-1, yrange.max()* np.sqrt(3) / 2+1)

    ax.axis('off')


In [5]:
def peakfinder(finaldf):

    peak_vector = []
    err = []
    
        
    for label in finaldf.index.unique():

        wavenumber = pd.Series(finaldf.columns.values)
        intensity = finaldf[finaldf.index==label]
        avg = intensity.mean().values
        avg = smooth2(7,  avg)
        avg = (avg - avg.min())/(avg.max()-avg.min())
       
       
        local_max_index = find_local_max(avg)
    
        stem = np.zeros(wavenumber.shape)
        stem[local_max_index] = 1  
        
        lorentz = []

        
        for wav in wavenumber[local_max_index]:
    
            reg = fit_region(wavenumber,wav,10)
            
            try:
                popt_1lorentz, pcov_1lorentz = scipy.optimize.curve_fit(_1Lorentzian, 
                                                                reg, 
                                                                avg[reg.index],
                                                                p0=[avg[reg.index].max(), wav,  2/(np.pi*avg[reg.index].max())])
                
                perr_1lorentz = np.sqrt(np.diag(pcov_1lorentz))
        
                pars_1 = popt_1lorentz[0:3]
                
                if (abs(pars_1[-1])>100) | (pars_1[0]<0):
                    pass
                else:
                    

        
                    lorentz.append(_1Lorentzian(wavenumber,*popt_1lorentz))
                    peak_vector.append([label]+popt_1lorentz.tolist()+['major'])
                    err.append(perr_1lorentz[0])
            
            except:
                pass
     
    for peak, e in zip(peak_vector,err):
        peak.append(e)
    
    peak = pd.DataFrame(peak_vector,columns=['label','height','center','width','importance','err'])
        
    return peak


In [6]:
def PCAPlot2D(df,target,info):  
    
    fig = plt.figure(figsize=(9,9/1.618))
    ax = fig.add_subplot(1,1,1)
    
    plt.xlabel(str(info[0]))
    plt.ylabel(str(info[1]))
    
    
    NUM_COLORS = len(list(np.unique(target)))  
    cmaps = plt.get_cmap('gist_rainbow')
    colours = [cmaps(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
        
    for label,color in zip(list(np.unique(target)),colours):
        indicesToKeep = df['sample'] == label
        
        if df.loc[indicesToKeep].shape[0] < 2:
            
            new_x = df.loc[indicesToKeep, 'principal component 1']
            new_y = df.loc[indicesToKeep, 'principal component 2']
            ax.scatter(new_x, new_y, s =100,alpha=1,label=label,marker='x',color = color)
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = OrderedDict(zip(labels, handles))
            plt.legend(loc='best',frameon=False)
            
        else:
                
          
            gmm = GaussianMixture(n_components=1).fit(pd.concat([df.loc[indicesToKeep, 'principal component 1']
                       , df.loc[indicesToKeep, 'principal component 2']],axis=1).values)
        
        
           
            for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
#                    print(pos)
#                    print(covar)
                
                
                if covar.shape == (2, 2):
                    U, s, Vt = np.linalg.svd(covar)
                    angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
                    width, height = 2 * np.sqrt(s)
                else:
                    angle = 0
                    width, height = 2 * np.sqrt(covar)
                    
            #draw the 2sigma region
                ax.add_patch(Ellipse(pos,2*width,2*height,angle,alpha=0.3,color = color))
        
                
                new_x = df.loc[indicesToKeep, 'principal component 1']
                new_y = df.loc[indicesToKeep, 'principal component 2']
                
            #    ax.scatter(new_x, new_y, c = color , s =100,alpha=1,label=target,marker='x')
                
                range1s = (((new_x < pos[0]+np.sqrt(width/2)) & (new_x > pos[0]-np.sqrt(width/2))) | 
                        ((new_y < pos[1]+np.sqrt(height/2)) & (new_y > pos[1]-np.sqrt(height/2))))
                ax.scatter(new_x[range1s], new_y[range1s], s =100,alpha=1,label=label,marker='x',color = color)
            
            #    plot_gmm(gmm)
            
                handles, labels = plt.gca().get_legend_handles_labels()
                by_label = OrderedDict(zip(labels, handles))
                plt.legend(loc='best',frameon=False)
                
    plt.show()

# Commands

In [7]:
# get data from files
spec,wave = analyse_data_folders(mainfolders,clean)

In [8]:
#The way the data has been organized to optimize the memory allocation has been: the index as the sample label
# i.e. the name of the folder. and the spectra in each row. At the end of the complete processing the header
# will have the wavenumber
spec.head()

,0,1,2,3,4,5,6,7,8,9,...,877,878,879,880,881,882,883,884,885,886
Corynebacterium glutamicum,0.227653,0.227494,0.223579,0.214980,0.227035,0.218605,0.219030,0.215026,0.195523,0.211218,...,0.430927,0.428915,0.400193,0.331481,0.297431,0.212763,0.173664,0.133377,0.070725,0.048149
Corynebacterium glutamicum,0.215113,0.215774,0.213217,0.205258,0.218897,0.210378,0.211386,0.207202,0.187301,0.204562,...,0.228752,0.249097,0.244289,0.200097,0.195867,0.141188,0.138251,0.137446,0.116980,0.143007
Corynebacterium glutamicum,0.227842,0.227642,0.223712,0.215094,0.227110,0.218647,0.219009,0.214930,0.195431,0.211156,...,0.429548,0.427670,0.399110,0.330512,0.296687,0.212218,0.173361,0.133343,0.070989,0.048778
Corynebacterium glutamicum,0.226069,0.220200,0.220987,0.212507,0.228932,0.215696,0.219596,0.214280,0.196440,0.215024,...,0.233244,0.250619,0.244987,0.204037,0.199275,0.143836,0.142151,0.139398,0.118353,0.143631
Corynebacterium glutamicum,0.238383,0.226446,0.228642,0.218920,0.236266,0.218702,0.224265,0.217446,0.199851,0.218921,...,0.238019,0.253590,0.247277,0.208225,0.202940,0.146026,0.145295,0.141102,0.119279,0.144374


In [9]:
# One thing to not related to the wavelength values is that, depending on the day of usage of the machine,
# the values will be sligtly shifted +/-0.01 cm-1. Still to make a proper analysis we need to interpolate them
# to the same wavelenght values.
wave.head()

,0,1,2,3,4,5,6,7,8,9,...,877,878,879,880,881,882,883,884,885,886
#Wave,1713.643555,1712.668945,1711.693359,1710.71875,1709.743164,1708.767578,1707.791992,1706.81543,1705.839844,1704.863281,...,770.880859,769.695313,768.508789,767.322266,766.134766,764.947266,763.759766,762.572266,761.383789,760.195313
#Wave,1713.643555,1712.668945,1711.693359,1710.71875,1709.743164,1708.767578,1707.791992,1706.81543,1705.839844,1704.863281,...,770.880859,769.695313,768.508789,767.322266,766.134766,764.947266,763.759766,762.572266,761.383789,760.195313
#Wave,1713.643555,1712.668945,1711.693359,1710.71875,1709.743164,1708.767578,1707.791992,1706.81543,1705.839844,1704.863281,...,770.880859,769.695313,768.508789,767.322266,766.134766,764.947266,763.759766,762.572266,761.383789,760.195313
#Wave,1713.643555,1712.668945,1711.693359,1710.71875,1709.743164,1708.767578,1707.791992,1706.81543,1705.839844,1704.863281,...,770.880859,769.695313,768.508789,767.322266,766.134766,764.947266,763.759766,762.572266,761.383789,760.195313
#Wave,1713.643555,1712.668945,1711.693359,1710.71875,1709.743164,1708.767578,1707.791992,1706.81543,1705.839844,1704.863281,...,770.880859,769.695313,768.508789,767.322266,766.134766,764.947266,763.759766,762.572266,761.383789,760.195313


In [10]:
# fix wavenumber among different datasets, ie, unify wavenumber
finaldf = fixwave(spec,wave)
finaldf.head()

KeyboardInterrupt: 

In [ ]:
#plot average
plot_avg(finaldf)
PlotAvgSpec(finaldf)

Note to save these images go to File > Download as > Markdown (.md). Then open .svg files in Inkscape to manually edit them

In [ ]:
#PCA analysis
#pca,wavenumber,df = pcafit(finaldf.values,finaldf.columns,finaldf.index)
df,target,info,pca,wavenumber = OrganizePCAData(finaldf.values, finaldf.index, finaldf.columns)

In [ ]:
PCAPlot2D(df,df['sample'],info)

In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.rcParamsDefault['axes.prop_cycle']
LoadingsPlot(wavenumber,pca)

In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.rcParamsDefault['axes.prop_cycle']
loadingpeak(finaldf)

In [ ]:
# find most important peaks using a inteligent lorentzina fit
#RUN FIRST THIS CODE AND THEN COMMENT IT AFTER THE .CSV FILE IS GENERATED!!
peaks = peakfinder(finaldf)
peaks.head()

In [ ]:
#Uncomment next line to save the peak file. Manually check the file and write your costum peak list
#peaks.to_csv(savefolder+'test.csv')

#Example of a peaks list were manually selected from the csv file created by the peakfinder function. write the peaks
# of interest that you found inside the brakets, uncomment the next line and run the 'get stats' script to make the
# histogram plot
peaks = [780, 850, 1000, 1155, 1210, 1335, 1450, 1525, 1575, 1610, 1660]

In [ ]:
#this script will run the different peaks in the previous peak list and chech the intensity with respect to all the
# different measurements and samples and thus buiding a statistical significance from the intensity of the peak.
# test='t-test_ind'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colours)
get_stats(finaldf,peaks)

In [ ]:
#SOM section insert the number of hexagon sides that you want (square shape)
som = LoadSOM(finaldf,neuron)
HexagonaSOMPlot(som,finaldf,neuron)